In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [49]:
data2 = pd.read_csv('Age_Specific_Coverage_Flu_RD1_2022_23_Sc_A_B_C_D.csv')


In [50]:
data2

,Geography,Age,Population,Week_Ending_Sat,Boost.coverage.rd1.sc_A_B,Boost.coverage.rd1.sc_C_D
0,Alabama,6 Months - 4 Years,263797,2022-08-06,0.000000,0.000000
1,Alabama,6 Months - 4 Years,263797,2022-08-13,0.000000,0.000000
2,Alabama,6 Months - 4 Years,263797,2022-08-20,2.601487,2.128490
3,Alabama,6 Months - 4 Years,263797,2022-08-27,3.442308,2.816434
4,Alabama,6 Months - 4 Years,263797,2022-09-03,4.912908,4.019652
...,...,...,...,...,...,...
16011,United States,65+ Years,383956567,2023-05-06,80.474001,65.842365
16012,United States,65+ Years,383956567,2023-05-13,80.731859,66.053339
16013,United States,65+ Years,383956567,2023-05-20,80.987051,66.262133
16014,United States,65+ Years,383956567,2023-05-27,81.229599,66.460581


In [51]:
data_date = pd.read_excel('date-vacc.xlsx')

In [52]:
date_series = data_date['date'].squeeze()
date_series_str = date_series.dt.strftime('%m/%d/%Y')

In [53]:
date_series_str

0      08/07/2022
1      08/08/2022
2      08/09/2022
3      08/10/2022
4      08/11/2022
          ...    
296    05/30/2023
297    05/31/2023
298    06/01/2023
299    06/02/2023
300    06/03/2023
Name: date, Length: 301, dtype: object

In [54]:
data_dem2 = pd.read_excel('demog.xlsx')

In [55]:
state_abv= pd.read_csv('states.csv')

In [56]:
def get_state(state, df2):
    df = df2[df2['State'] == state]
    a = df['Abbreviation']
    if df.size == 0:
        return float('nan')
    else:
        return a.squeeze()

In [57]:
get_state('United States' , state_abv)

'US'

In [58]:
def get_state2(state, df2):
    df = df2[df2['Abbreviation'] == state]
    a = df['State']
    if df.size == 0:
        return float('nan')
    else:
        return a.squeeze()

In [59]:
def get_pop(state, df2, b):
    state1=get_state(state, state_abv)
    #print(state1)
    df = df2[df2['state'] == state1] 
    a = df[b]
    if df.size == 0:
        return float('nan')
    else:
        return a.squeeze()

In [60]:
data_date2 = pd.read_excel('date-zerovacc.xlsx')
date_series2 = data_date2['date'].squeeze()
date_series_str2 = date_series2.dt.strftime('%m/%d/%Y')

In [61]:
date_series_str2

0      04/22/2022
1      04/23/2022
2      04/24/2022
3      04/25/2022
4      04/26/2022
          ...    
103    08/03/2022
104    08/04/2022
105    08/05/2022
106    08/06/2022
107    08/07/2022
Name: date, Length: 108, dtype: object

In [62]:
def baseline(state):
    doses_series = pd.DataFrame(columns = ['date', '0_4','5_12','13_17','18_49','50_64','65+'])
    
    
    selected=data2[data2['Geography']==state]
    selected_0to4=selected[selected['Age']=='6 Months - 4 Years']
    selected_5to12=selected[selected['Age']=='5-12 Years']
    selected_13to17=selected[selected['Age']=='13-17 Years']
    selected_18to49=selected[selected['Age']=='18-49 Years']
    selected_50to64=selected[selected['Age']=='50-64 Years']
    selected_65more=selected[selected['Age']=='65+ Years']
    
    for i in range(1, len(selected_0to4)):
        diff_0to4CD=float((selected_0to4.iloc[i,5]-selected_0to4.iloc[i-1,5])/700)*get_pop(state, data_dem2, '0_4')*1.05
        #0-4, by the end of may,63.9% (62.4%-65.3%)
        diff_5to12CD=float((selected_5to12.iloc[i,5]-selected_5to12.iloc[i-1,5])/700)*get_pop(state, data_dem2, '5_11')*1.076
        #5-12, by the end of may,57.1% (56.1%-58.2%)
        diff_13to17CD=float((selected_13to17.iloc[i,5]-selected_13to17.iloc[i-1,5])/700)*get_pop(state, data_dem2, '12_18')*1.048
        #13-17, by the end of may,48% (46.7%-49.3%)
        diff_18to49CD=float((selected_18to49.iloc[i,5]-selected_18to49.iloc[i-1,5])/700)*get_pop(state, data_dem2, '19_49')*1.074
        #18-49, by the end of may,35.8% (34.8%-36.8%)
        diff_50to64CD=float((selected_50to64.iloc[i,5]-selected_50to64.iloc[i-1,5])/700)*get_pop(state, data_dem2, '50_64')*1.063
        #50-64, by the end of may,50.3% (48.8%-51.8%)
        diff_65CD=float((selected_65more.iloc[i,5]-selected_65more.iloc[i-1,5])/700)*get_pop(state, data_dem2, '65+')*1.026
        #65+, by the end of may, 68.4% (67%-69.8%)
        for j in range(0,7):
            date=date_series_str[(7*(i-1)+j)]
            doses_series = doses_series.append({'date' : date, '0_4' :  max(0,diff_0to4CD), '5_12' : max(0,diff_5to12CD), '13_17' : max(0,diff_13to17CD), '18_49': max(0,diff_18to49CD), '50_64': max(0,diff_50to64CD), '65+': max(0,diff_65CD)}, 
                    ignore_index = True)
    
    return doses_series

In [63]:
baseline('United States')

,date,0_4,5_12,13_17,18_49,50_64,65+
0,08/07/2022,0,0,0,0,0,0
1,08/08/2022,0,0,0,0,0,0
2,08/09/2022,0,0,0,0,0,0
3,08/10/2022,0,0,0,0,0,0
4,08/11/2022,0,0,0,0,0,0
...,...,...,...,...,...,...,...
296,05/30/2023,4747.373127,1824.482253,2302.190648,43277.015581,13588.386518,13921.418943
297,05/31/2023,4747.373127,1824.482253,2302.190648,43277.015581,13588.386518,13921.418943
298,06/01/2023,4747.373127,1824.482253,2302.190648,43277.015581,13588.386518,13921.418943
299,06/02/2023,4747.373127,1824.482253,2302.190648,43277.015581,13588.386518,13921.418943


In [64]:
def target(state):
    doses_series = pd.DataFrame(columns = ['date', '0_4','5_12','13_17','18_49','50_64','65+'])
    
    
    selected=data2[data2['Geography']==state]
    selected_0to4=selected[selected['Age']=='6 Months - 4 Years']
    selected_5to12=selected[selected['Age']=='5-12 Years']
    selected_13to17=selected[selected['Age']=='13-17 Years']
    selected_18to49=selected[selected['Age']=='18-49 Years']
    selected_50to64=selected[selected['Age']=='50-64 Years']
    selected_65more=selected[selected['Age']=='65+ Years']
    
    for i in range(1, len(selected_0to4)):
        diff_0to4CD=float((selected_0to4.iloc[i,5]-selected_0to4.iloc[i-1,5])/700)*get_pop(state, data_dem2, '0_4')*(70/selected_0to4.iloc[len(selected_0to4)-1,5])
        #0-4, by the end of may,63.9% (62.4%-65.3%)
        diff_5to12CD=float((selected_5to12.iloc[i,5]-selected_5to12.iloc[i-1,5])/700)*get_pop(state, data_dem2, '5_11')*(70/selected_5to12.iloc[len(selected_0to4)-1,5])
        #5-12, by the end of may,57.1% (56.1%-58.2%)
        diff_13to17CD=float((selected_13to17.iloc[i,5]-selected_13to17.iloc[i-1,5])/700)*get_pop(state, data_dem2, '12_18')*(70/selected_13to17.iloc[len(selected_0to4)-1,5])
        #13-17, by the end of may,48% (46.7%-49.3%)
        diff_18to49CD=float((selected_18to49.iloc[i,5]-selected_18to49.iloc[i-1,5])/700)*get_pop(state, data_dem2, '19_49')*(70/selected_18to49.iloc[len(selected_0to4)-1,5])
        #18-49, by the end of may,35.8% (34.8%-36.8%)
        diff_50to64CD=float((selected_50to64.iloc[i,5]-selected_50to64.iloc[i-1,5])/700)*get_pop(state, data_dem2, '50_64')*(70/selected_50to64.iloc[len(selected_0to4)-1,5])
        #50-64, by the end of may,50.3% (48.8%-51.8%)
        diff_65CD=float((selected_65more.iloc[i,5]-selected_65more.iloc[i-1,5])/700)*get_pop(state, data_dem2, '65+')*(70/selected_65more.iloc[len(selected_0to4)-1,5])
        #65+, by the end of may, 68.4% (67%-69.8%)
        for j in range(0,7):
            date=date_series_str[(7*(i-1)+j)]
            doses_series = doses_series.append({'date' : date, '0_4' :  max(0,diff_0to4CD), '5_12' : max(0,diff_5to12CD), '13_17' : max(0,diff_13to17CD), '18_49': max(0,diff_18to49CD), '50_64': max(0,diff_50to64CD), '65+': max(0,diff_65CD)}, 
                    ignore_index = True)
    
    return doses_series

In [65]:
target('United States')

,date,0_4,5_12,13_17,18_49,50_64,65+
0,08/07/2022,0,0,0,0,0,0
1,08/08/2022,0,0,0,0,0,0
2,08/09/2022,0,0,0,0,0,0
3,08/10/2022,0,0,0,0,0,0
4,08/11/2022,0,0,0,0,0,0
...,...,...,...,...,...,...,...
296,05/30/2023,5200.20666,2237.676093,3356.991697,84585.386923,18721.647325,14252.655625
297,05/31/2023,5200.20666,2237.676093,3356.991697,84585.386923,18721.647325,14252.655625
298,06/01/2023,5200.20666,2237.676093,3356.991697,84585.386923,18721.647325,14252.655625
299,06/02/2023,5200.20666,2237.676093,3356.991697,84585.386923,18721.647325,14252.655625


In [66]:
vacc_base = baseline('United States')
y = "vacc_base.csv"
vacc_base.to_csv(y)
vacc_target = target('United States')
y2 = "vacc_target.csv"
vacc_target.to_csv(y2)

In [67]:
print("0_4")
print(vacc_base['0_4'].sum()/(get_pop('United States', data_dem2, '0_4')))
print(vacc_target['0_4'].sum()/(get_pop('United States', data_dem2, '0_4')))
print("5_12")
print(vacc_base['5_12'].sum()/(get_pop('United States', data_dem2, '5_11')))
print(vacc_target['5_12'].sum()/(get_pop('United States', data_dem2, '5_11')))
print("13_17")
print(vacc_base['13_17'].sum()/(get_pop('United States', data_dem2, '12_18')))
print(vacc_target['13_17'].sum()/(get_pop('United States', data_dem2, '12_18')))
print("18_49")
print(vacc_base['18_49'].sum()/(get_pop('United States', data_dem2, '19_49')))
print(vacc_target['18_49'].sum()/(get_pop('United States', data_dem2, '19_49')))
print("50_64")
print(vacc_base['50_64'].sum()/(get_pop('United States', data_dem2, '50_64')))
print(vacc_target['50_64'].sum()/(get_pop('United States', data_dem2, '50_64')))
print("65+")
print(vacc_base['65+'].sum()/(get_pop('United States', data_dem2, '65+')))
print(vacc_target['65+'].sum()/(get_pop('United States', data_dem2, '65+')))


0_4
0.639044062417694
0.7000000000000003
5_12
0.5707428260398181
0.7000000000000013
13_17
0.4800528566148448
0.6999999999999998
18_49
0.3592035838266672
0.7020672223902235
50_64
0.509225700112668
0.7015949946498872
65+
0.6837317561285186
0.6999999999999982


In [78]:
(get_pop('United States', data_dem2, '0_4')+(get_pop('United States', data_dem2, '5_11'))+(get_pop('United States', data_dem2, '12_18')))*0.1395

10505452.05

In [25]:
state_list = pd.read_excel('FIPState.xlsx')

In [26]:
state_series = state_list['st'].squeeze()

In [27]:
state_series

0     AL 
1     AK 
2     AZ 
3     AR 
4     CA 
5     CO 
6     CT 
7     DE 
8     FL 
9     GA 
10    HI 
11    ID 
12    IL 
13    IN 
14    IA 
15    KS 
16    KY 
17    LA 
18    ME 
19    MD 
20    MA 
21    MI 
22    MN 
23    MS 
24    MO 
25    MT 
26    NE 
27    NV 
28    NH 
29    NJ 
30    NM 
31    NY 
32    NC 
33    ND 
34    OH 
35    OK 
36    OR 
37    PA 
38    RI 
39    SC 
40    SD 
41    TN 
42    TX 
43    UT 
44    VT 
45    VA 
46    WA 
47    WV 
48    WI 
49    WY 
Name: st, dtype: object

In [28]:
listo = list(state_series)

In [253]:
listo = list(state_series)
for x in listo:
    x = x.strip()
    xname=get_state2(x , state_abv)
    vacc_dataAB = primaryAB(xname)
    y = "vacc_data/vac_" + x + "_AB.csv"
    vacc_dataAB.to_csv(y)
    vacc_dataCD = primaryCD(xname)
    y2 = "vacc_data/vac_" + x + "_CD.csv"
    vacc_dataCD.to_csv(y2)

In [35]:
final_doses_AB = pd.DataFrame(columns = ['state', '0_4','5_12','13_17','18_49','50_64','65+'])
final_doses_CD = pd.DataFrame(columns = ['state', '0_4','5_12','13_17','18_49','50_64','65+'])
for x in listo:
    x = x.strip()
    xname=get_state2(x , state_abv)
    selected=data2[data2['Geography']==xname]
    selected_0to4=selected[selected['Age']=='6 Months - 4 Years']
    selected_5to12=selected[selected['Age']=='5-12 Years']
    selected_13to17=selected[selected['Age']=='13-17 Years']
    selected_18to49=selected[selected['Age']=='18-49 Years']
    selected_50to64=selected[selected['Age']=='50-64 Years']
    selected_65more=selected[selected['Age']=='65+ Years']
    final_doses_AB = final_doses_AB.append({'state' : x, '0_4' :  selected_0to4.iloc[len(selected_0to4)-1,4], '5_12' : selected_5to12.iloc[len(selected_0to4)-1,4], '13_17' : selected_13to17.iloc[len(selected_0to4)-1,4], '18_49': selected_18to49.iloc[len(selected_0to4)-1,4], '50_64': selected_50to64.iloc[len(selected_0to4)-1,4], '65+': selected_65more.iloc[len(selected_0to4)-1,4]}, 
                    ignore_index = True)
    final_doses_CD = final_doses_CD.append({'state' : x, '0_4' :  selected_0to4.iloc[len(selected_0to4)-1,5], '5_12' : selected_5to12.iloc[len(selected_0to4)-1,5], '13_17' : selected_13to17.iloc[len(selected_0to4)-1,5], '18_49': selected_18to49.iloc[len(selected_0to4)-1,5], '50_64': selected_50to64.iloc[len(selected_0to4)-1,5], '65+': selected_65more.iloc[len(selected_0to4)-1,5]}, 
                    ignore_index = True)
x='US'
x = x.strip()
xname=get_state2(x , state_abv)
selected=data2[data2['Geography']==xname]
selected_0to4=selected[selected['Age']=='6 Months - 4 Years']
selected_5to12=selected[selected['Age']=='5-12 Years']
selected_13to17=selected[selected['Age']=='13-17 Years']
selected_18to49=selected[selected['Age']=='18-49 Years']
selected_50to64=selected[selected['Age']=='50-64 Years']
selected_65more=selected[selected['Age']=='65+ Years']
final_doses_AB = final_doses_AB.append({'state' : x, '0_4' :  selected_0to4.iloc[len(selected_0to4)-1,4], '5_12' : selected_5to12.iloc[len(selected_0to4)-1,4], '13_17' : selected_13to17.iloc[len(selected_0to4)-1,4], '18_49': selected_18to49.iloc[len(selected_0to4)-1,4], '50_64': selected_50to64.iloc[len(selected_0to4)-1,4], '65+': selected_65more.iloc[len(selected_0to4)-1,4]}, 
                ignore_index = True)
final_doses_CD = final_doses_CD.append({'state' : x, '0_4' :  selected_0to4.iloc[len(selected_0to4)-1,5], '5_12' : selected_5to12.iloc[len(selected_0to4)-1,5], '13_17' : selected_13to17.iloc[len(selected_0to4)-1,5], '18_49': selected_18to49.iloc[len(selected_0to4)-1,5], '50_64': selected_50to64.iloc[len(selected_0to4)-1,5], '65+': selected_65more.iloc[len(selected_0to4)-1,5]}, 
                ignore_index = True)
y3 = "cumulative_dose_AB.csv"
y4 = "cumulative_dose_CD.csv"
final_doses_AB.to_csv(y3)
final_doses_CD.to_csv(y4)

In [30]:
final_doses = pd.DataFrame(columns = ['state', '0_4','5_12','13_17','18_49','50_64','65+'])

for x in listo:
    x = x.strip()
    xname=get_state2(x , state_abv)
    selected=data2[data2['Geography']==xname]
    selected_0to4=selected[selected['Age']=='6 Months - 4 Years']
    selected_5to12=selected[selected['Age']=='5-12 Years']
    selected_13to17=selected[selected['Age']=='13-17 Years']
    selected_18to49=selected[selected['Age']=='18-49 Years']
    selected_50to64=selected[selected['Age']=='50-64 Years']
    selected_65more=selected[selected['Age']=='65+ Years']
    final_doses = final_doses.append({'state' : x, '0_4' :  float(selected_0to4.iloc[len(selected_0to4)-1,4])/1.1, '5_12' : float(selected_5to12.iloc[len(selected_0to4)-1,4])/1.1, '13_17' : float(selected_13to17.iloc[len(selected_0to4)-1,4])/1.1, '18_49': float(selected_18to49.iloc[len(selected_0to4)-1,4])/1.1, '50_64': float(selected_50to64.iloc[len(selected_0to4)-1,4])/1.1, '65+': float(selected_65more.iloc[len(selected_0to4)-1,4])/1.1}, 
                    ignore_index = True)

x='US'
x = x.strip()
xname=get_state2(x , state_abv)
selected=data2[data2['Geography']==xname]
selected_0to4=selected[selected['Age']=='6 Months - 4 Years']
selected_5to12=selected[selected['Age']=='5-12 Years']
selected_13to17=selected[selected['Age']=='13-17 Years']
selected_18to49=selected[selected['Age']=='18-49 Years']
selected_50to64=selected[selected['Age']=='50-64 Years']
selected_65more=selected[selected['Age']=='65+ Years']
final_doses = final_doses.append({'state' : x, '0_4' :  float(selected_0to4.iloc[len(selected_0to4)-1,4])/1.1, '5_12' : float(selected_5to12.iloc[len(selected_0to4)-1,4])/1.1, '13_17' : float(selected_13to17.iloc[len(selected_0to4)-1,4])/1.1, '18_49': float(selected_18to49.iloc[len(selected_0to4)-1,4])/1.1, '50_64': float(selected_50to64.iloc[len(selected_0to4)-1,4])/1.1, '65+': float(selected_65more.iloc[len(selected_0to4)-1,4])/1.1}, 
                    ignore_index = True)
y5 = "cumulative_dose.csv"

final_doses.to_csv(y5)
